**Assignment 2**
----------------------------------
**Name** : *Ahmad Farhan* <br>
**Roll No.** : *i211366* <br>
**Section** : *A* <br>
__________________________________

### *Heuristic Backtracking Solution to Sudoku Problem*

#### Heuristics
1. Minimum Remaining Value(MRV)<br>
MRV value of a cell is the size of its remaining domain<br>
Pick Variable with minimum MRV Value

2. Degree Heuristic<br>
Degree value is the number of empty peers that a cell has<br>
Pick Variable with minimum Degree Value

3. Least Constraining Score(LCV)<br>
LCS of a value is the number of peer cell domains it belongs to<br>
Apply Values in increasing order of LCS Value

#### Algorithm Analysis
Time Complexity:$O(b^d)$ <br>
Space Complexity: $O(n+d)$ <br>
- n is Number of cells 
- b is Avg number of varaible choices
- d is Depth of recursion (number of empty cells)

In [1]:
import numpy as np

In [2]:
# Function to calculate the domain of a row or column
def rc_domain(board, n, row=False):
    available = np.zeros(10,dtype=int)
    for i in range(9):
        if row: available[board[n][i]]=1
        else: available[board[i][n]]=1
    return available

# Function to calculate the domain of a 3x3 block
def block_domain(board, row, col):
    available = np.zeros(10,dtype=int)
    for i in range(3):
        for j in range(3):
            available[board[row+i][col+j]] = 1
    return available

# i,j must be (0,3,6) 
# Function to calculate the hash value for a 3x3 block
def bhash(i,j): return int(i+j/3)+18

# i,j may be any[0-8] 
# Function to calculate the hash value for a 3x3 block
def block(i,j): return i//3*3+j//3 + 18

# Function to convert linear index to row-column position
def lintopos(linear): return linear//9, linear%9

# Function to get top-left corner of block
def grid_idx(row, col): return row//3*3, col//3*3

# Function to calculate the hash table
def cal_hash(sudoku):
    hash = {}
    for i in range(9): hash[i] = rc_domain(sudoku, i, True) # Row Domains
    for i in range(9): hash[i+9]= rc_domain(sudoku, i)   # Column Domains
    for i in range(0,9,3):
        for j in range(0,9,3):
            hash[bhash(i,j)] = block_domain(sudoku, i, j)  # Block Domains
    return hash

# Function to update the hash table after assigning a value to a cell
def update_hash(hash, val, row, col, mark):
    hash[row][val] = mark
    hash[col+9][val] = mark
    hash[block(row,col)][val] = mark

# Function to calculate the domain of a cell based on the hash table
def cal_domain(hash, row, col):
    rh = hash[row]; ch = hash[col+9]; bh = hash[block(row,col)]
    domain = [val for val, t in enumerate(zip(rh, ch, bh)) if not any(t)]
    return domain

# Function to calculate domains for all cells
def cal_domains(board, hash):
    domains = []
    for i in range(9):
        row = [cal_domain(hash, i, j) if board[i][j] == 0
                else [] for j in range(9)]
        domains.append(row)
    return domains

# Function to remove a value from a list if it exists
def remove_val(lst, value):
    try: lst.remove(value)
    except ValueError: pass

# Function to extract cells with minimum domain length from an array
def extract_min(array):
    i = 0; tmp_list = []
    while array[i][2] == array[0][2]: 
        tmp_list.append(array[i]);i+=1
        if i >= len(array):break
    return tmp_list


In [3]:
# Function to use Pre-defined Test Boards
def gboard(i=0):
  if i == 0:
    return [[0,0,0,4,0,0,5,0,6],
            [0,0,0,0,0,0,4,3,0],
            [0,6,0,5,0,0,0,1,0],
            [0,0,3,0,5,0,0,9,0],
            [4,0,7,0,0,0,8,0,0],
            [9,0,0,0,8,7,2,0,0],
            [0,7,1,0,0,5,0,0,0],
            [0,0,0,0,0,6,0,0,0],
            [0,5,0,0,0,1,0,0,4]]
  elif i ==1: 
    return [[0,0,0,0,0,0,0,0,0],
            [3,0,2,5,4,0,0,0,0],
            [0,5,0,3,0,1,0,7,0],
            [0,0,0,0,0,0,0,0,4],
            [4,0,9,0,0,6,0,0,5],
            [0,2,3,0,5,4,7,9,0],
            [0,0,0,0,0,0,0,5,0],
            [7,0,0,8,1,0,0,0,0],
            [0,8,0,0,6,0,0,0,9]]
  elif i == 2: 
    return [[0,6,0,1,0,4,0,5,0],
            [0,0,8,3,0,5,6,0,0],
            [2,0,0,0,0,0,0,0,1],
            [8,0,0,4,0,7,0,0,6],
            [0,0,6,0,0,0,3,0,0],
            [7,0,0,9,0,1,0,0,4],
            [5,0,0,0,0,0,0,0,2],
            [0,0,7,2,0,6,9,0,0],
            [0,4,0,5,0,8,0,7,0]]
  else:
    return [[5,3,0,0,7,0,0,0,0],
            [6,0,0,1,9,5,0,0,0],
            [0,9,8,0,0,0,0,6,0],
            [8,0,0,0,6,0,0,0,3],
            [4,0,0,8,0,3,0,0,1],
            [7,0,0,0,2,0,0,0,6],
            [0,6,0,0,0,0,2,8,0],
            [0,0,0,4,1,9,0,0,5],
            [0,0,0,0,8,0,0,7,9]]

# Function to randomly clear values from the Sudoku board to create a puzzle
def clear_values(board,n_zeros):
    for _ in range(n_zeros):
        while True:
            row = np.random.randint(0, 8)
            col = np.random.randint(0, 8)
            if board[row][col] != 0:
                board[row][col] = 0
                break

# Function to select first empty cell
def select_empty(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0: 
                return i,j
    return 0

# Function to check if a value is used in the row, column, or block
def val_used(hash, row, col, value):
    rh = hash[row]; ch = hash[col+9]; bh = hash[block(row,col)]
    return rh[value] or ch[value] or bh[value]

# Function to fill the Sudoku board recursively using backtracking
nums_list = list(range(1,10))
def fill_board(board, hash):
    position = select_empty(board)          # Select First empty positon
    if position == 0: return True           # If no Empty Found, board complete
    
    r,c = position
    np.random.shuffle(nums_list)            # Randomize ordering of values
    for val in nums_list:
        if not val_used(hash, r, c, val):   # If value not used
            board[r][c] = val
            update_hash(hash, val, r, c, 1)     # Apply Value and explore forward
            if fill_board(board, hash): return True
            update_hash(hash, val, r, c, 0)     # If value didnt lead to solution, backtrack
    board[r][c] = 0

# Function to generate a Sudoku puzzle
def generate_sudoku(n_zeros=40):
    #board = np.zeros((9,9), dtype=int)
    board = [[0]*9 for _ in range(9)]       # Initialize an Empty Board
    hash = cal_hash(board)                  # Initialize its Hash Table
    fill_board(board,hash)                  # Consistently Fill board with values
    clear_values(board,n_zeros)             # Clear n values from board
    return board

In [4]:
# Function to find cells with domain size of one
def argonedomain(domains):
    positions = []
    for i in range(9):
        for j in range(9):
            if len(domains[i][j]) == 1:
                positions.append((i,j,domains[i][j][0]))
    return positions

# Function to update cells with domain size one
def updateones(game, positions):
    for r,c,val in positions:
        game.board[r][c] = val
        game.forwardupdate(r, c, val)
        game.filled += 1

# Function to update all cells with single domain values
def preprocess(game):
    positions = argonedomain(game.domains)
    while (len(positions) > 0):
        updateones(game, positions)
        positions = argonedomain(game.domains)

# Function to check if a group (row, column, or subgrid) is valid (no repeated values)
def is_valid_group(group):
    group = group[group != 0]       # Remove zeros 
    return len(np.unique(group)) == len(group)  # Check all values unqiue

# Function to verify if a completed Sudoku board is valid
def verify(sudoku):
    sudoku = np.array(sudoku)
    for row in sudoku:      # Check rows
        if not is_valid_group(row):
            return False
    for col in sudoku.T:    # Check Columns
        if not is_valid_group(col):
            return False
    for i in range(0, 9, 3):# Check Blocks
        for j in range(0, 9, 3):
            subgrid = sudoku[i:i+3, j:j+3]
            if not is_valid_group(subgrid.flatten()):
                return False    
    return True

### Heuristics

In [5]:
# Minimum Remaining Value Heuristic of cell
# Size of its remaining domain
def mrv(domains, r, c):
    return r,c,len(domains[r][c])

# Degree Heuristic of a cell
# Number of empty cells in its row, column, and subgrid)
def deg(board, r, c):
    if board[r][c] != 0: return 0
    ur,uc = grid_idx(r,c); count = 0
    for i in range(3):
        for j in range(3):
            if board[ur+i][uc+j] == 0: count += 1
    for i in range(9):
        if (i < uc or i > uc+2) and board[r][i] == 0: count += 1
        if (i < ur or i > ur+2) and board[i][c] == 0: count += 1
    return r,c,count

# Least Constraining Value (LCS) Heuristic for a cell
# No. of peer cell domains that a value is found in
def lcs(domains, r, c, val):
    ur,uc = grid_idx(r,c); count = 0
    for i in range(3):
        for j in range(3):
            if val in domains[ur+i][uc+j]: count += 1
    for i in range(9):
        if (i < ur or i > ur+2) and val in domains[r][i]: count += 1
        if (i < uc or i > uc+2) and val in domains[i][c]: count += 1
    return count

# Function to select the cell with the Minimum Remaining Value (MRV)
def select_mrv(board, domains):
    mrv_list = []
    for r in range(9):
        mrv_list.extend(mrv(domains,r,c) for c in range(9) if board[r][c] == 0)
    return min(mrv_list, key=lambda x: x[2])

# Function to select the cell with the highest degree
def select_deg(board, domains=None):
    deg_list = []
    for r in range(9):
        deg_list.extend(deg(board,r,c) for c in range(9) if board[r][c] == 0)
    return min(deg_list, key=lambda x: x[2])

# Function to select the cell with the Minimum Remaining Value (MRV) and highest degree (Degree tiebreaker)
def select_mrvdeg(board, domains):
    mrv_list = []; deg_list = []
    for r in range(9): mrv_list.extend(mrv(domains,r,c) for c in range(9) if board[r][c]==0)

    if min(mrv_list, key=lambda x:x[2])[2] == 0: return -1,-1,0
    mrv_list = sorted(mrv_list, key=lambda x: x[2])
    mrv_list = extract_min(mrv_list) 

    for r,c,m in mrv_list: 
        if board[r][c] == 0: deg_list.append(deg(board,r,c))
    if not deg_list: return -1,-1,0
    return min(deg_list, key=lambda x: x[2])

# Function to select the cell with the highest degree (Degree tiebreaker) and Minimum Remaining Value (MRV tiebreaker)
def select_degmrv(board, domains):
    mrv_list = []; deg_list = []
    for r in range(9): deg_list.extend(deg(board,r,c) for c in range(9) if board[r][c] == 0)

    if min(deg_list, key=lambda x:x[2])[2] == 0: return -1,-1,0
    deg_list = sorted(deg_list, key=lambda x: x[2])
    deg_list = extract_min(deg_list) 

    for r,c,m in deg_list: 
        if board[r][c] == 0: mrv_list.append(mrv(domains,r,c))
    if not mrv_list: return -1,-1,0
    return min(mrv_list, key=lambda x: x[2])

# Function to select all possible values for a cell
def select_all(domains, r, c): 
    return [(val,0) for val in domains[r][c]]

# Function to order domain values for a cell based on LCS heuristic
def select_lcs(domains, r, c):
    lcs_list = []
    for val in domains[r][c]:
        lcs_list.append((val, lcs(domains, r, c, val)))
    lcs_list = sorted(lcs_list, key=lambda x: x[1])
    return lcs_list

### Backtracking

In [6]:
# Sudoku class for representing and solving Sudoku puzzles
class Sudoku:
    def __init__(self, board):
        self.board = np.array(board).tolist() #board.copy()             # Sudoku Board
        self.hash = cal_hash(board)     # Hash representing domain constraints
        self.domains = cal_domains(board,self.hash) # Domain of each cell
        self.filled = np.count_nonzero(board)       # Number of filled cells

    # Method to display the Sudoku board  
    def display(self):
        for i in range(9):
            print(self.board[i])
        print()

    # Method to display the hash table and domains
    def displaymeta(self, h='h'):
        if h == 'h' or h == 'b':
            for key in self.hash.keys():
                print(key, hash[key])
        if h == 'd' or h == 'b': 
            for dom in self.domains: 
                print(dom)
    
    # Method to update the board and domains when a cell is filled forward
    def forwardupdate(self, r, c, value):
        self.board[r][c] = value    # Update cell value
        self.domains[r][c] = []     # Clear Comain for current cell
        ur, uc = grid_idx(r, c)     # Get top-left corner of block
        update_hash(self.hash, value, r, c, 1)  # Update Hash Table
        # Remove the value from domain of cells in the same row, column, and subgrid
        for i in range(3):
            for j in range(3):
                remove_val(self.domains[ur+i][uc+j],value)
        for i in range(9):
            if i < uc or i > uc+2: remove_val(self.domains[r][i],value)
            if i < ur or i > ur+2: remove_val(self.domains[i][c],value)

    # Method to update the board and domains when a cell is filled backward (backtracking)
    def backwardupdate(self, r, c, value):
        self.board[r][c] = 0        # Reset cell value
        ur, uc = grid_idx(r, c)     # Get top-left corner of block
        update_hash(self.hash, value, r, c, 0)  # Update Hash Table
        # Restore the domain of empty cells in the same row, column and subgrid
        for i in range(3):
            for j in range(3):
                if(self.board[ur+i][uc+j] == 0):
                    self.domains[ur+i][uc+j] = cal_domain(self.hash, ur+i, uc+j)
        for i in range(9):
            if(i < uc or i > uc+2) and self.board[r][i] == 0:
                self.domains[r][i] = cal_domain(self.hash, r, i)
            if(i < ur or i > ur+2) and self.board[i][c] == 0:
                self.domains[i][c] = cal_domain(self.hash, i, c)

    # Method to check if the board is solvable
    def issolvable(self):
        for i in range(9):
            for j in range(9):
                # If cell is empty and domain is empty, then not solvable
                if self.board[i][j] == 0 and len(self.domains[i][j]) == 0:
                    return False
        return True

    # Method to explore possible solutions using backtracking
    def explore(self, select_var, select_val):
        if self.filled == 9*9: return True      # If all cells are filled
        # Select Variable Using Heuristic
        row, col, mrv_val = select_var(self.board, self.domains)    
        if mrv_val == 0: return False   # If heuristic is zero, then backtrack

        self.filled += 1    
        values = select_val(self.domains, row, col) # Order of values fo variable
        for value, *_ in values:
            self.forwardupdate(row, col, value)     # Fill cell with value, update ds
            if self.explore(select_var,select_val): return True
            self.backwardupdate(row, col, value)    # Backtrack if no solution found
        self.filled -= 1
        return False

    # Method to solve the Sudoku using specified variable and value selection heuristics
    def solve(self, var_selector, val_selector):
        preprocess(self)
        if self.issolvable() and self.explore(var_selector, val_selector):
            print("Solution Board: ")
            self.display()
            print("Verifying Consistency :",verify(self.board))
        else: print("Sudoku not Solvable by Chosen Heuristics")


In [7]:
grid = generate_sudoku()

In [8]:
# Solving with MRV and LCS Heuristic
game = Sudoku(grid)
print("Initial Board: ")
game.display()
game.solve(select_mrv, select_lcs)

Initial Board: 
[0, 0, 0, 0, 0, 0, 0, 3, 4]
[0, 0, 1, 8, 0, 0, 0, 0, 7]
[0, 7, 9, 0, 0, 2, 0, 0, 6]
[0, 4, 6, 0, 3, 8, 0, 2, 5]
[8, 5, 0, 0, 0, 0, 0, 7, 1]
[2, 0, 7, 1, 5, 0, 3, 0, 8]
[0, 0, 0, 5, 0, 0, 7, 1, 2]
[0, 1, 0, 0, 0, 0, 6, 0, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Solution Board: 
[5, 2, 8, 6, 7, 9, 1, 3, 4]
[6, 3, 1, 8, 4, 5, 2, 9, 7]
[4, 7, 9, 3, 1, 2, 8, 5, 6]
[1, 4, 6, 7, 3, 8, 9, 2, 5]
[8, 5, 3, 9, 2, 6, 4, 7, 1]
[2, 9, 7, 1, 5, 4, 3, 6, 8]
[9, 8, 4, 5, 6, 3, 7, 1, 2]
[3, 1, 5, 2, 8, 7, 6, 4, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Verifying Consistency : True


In [9]:
# Solving with MRV, Degree and LCS Heuristic
game = Sudoku(grid)
print("Initial Board: ")
game.display()
game.solve(select_mrvdeg, select_lcs)

Initial Board: 
[0, 0, 0, 0, 0, 0, 0, 3, 4]
[0, 0, 1, 8, 0, 0, 0, 0, 7]
[0, 7, 9, 0, 0, 2, 0, 0, 6]
[0, 4, 6, 0, 3, 8, 0, 2, 5]
[8, 5, 0, 0, 0, 0, 0, 7, 1]
[2, 0, 7, 1, 5, 0, 3, 0, 8]
[0, 0, 0, 5, 0, 0, 7, 1, 2]
[0, 1, 0, 0, 0, 0, 6, 0, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Solution Board: 
[6, 2, 8, 9, 7, 5, 1, 3, 4]
[5, 3, 1, 8, 4, 6, 2, 9, 7]
[4, 7, 9, 3, 1, 2, 8, 5, 6]
[1, 4, 6, 7, 3, 8, 9, 2, 5]
[8, 5, 3, 6, 2, 9, 4, 7, 1]
[2, 9, 7, 1, 5, 4, 3, 6, 8]
[9, 8, 4, 5, 6, 3, 7, 1, 2]
[3, 1, 5, 2, 8, 7, 6, 4, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Verifying Consistency : True


In [10]:
# Solving with Degree and LCS Heuristic
game = Sudoku(grid)
print("Initial Board: ")
game.display()
game.solve(select_deg, select_lcs)

Initial Board: 
[0, 0, 0, 0, 0, 0, 0, 3, 4]
[0, 0, 1, 8, 0, 0, 0, 0, 7]
[0, 7, 9, 0, 0, 2, 0, 0, 6]
[0, 4, 6, 0, 3, 8, 0, 2, 5]
[8, 5, 0, 0, 0, 0, 0, 7, 1]
[2, 0, 7, 1, 5, 0, 3, 0, 8]
[0, 0, 0, 5, 0, 0, 7, 1, 2]
[0, 1, 0, 0, 0, 0, 6, 0, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Solution Board: 
[6, 2, 8, 9, 7, 5, 1, 3, 4]
[5, 3, 1, 8, 4, 6, 2, 9, 7]
[4, 7, 9, 3, 1, 2, 8, 5, 6]
[1, 4, 6, 7, 3, 8, 9, 2, 5]
[8, 5, 3, 6, 2, 9, 4, 7, 1]
[2, 9, 7, 1, 5, 4, 3, 6, 8]
[9, 8, 4, 5, 6, 3, 7, 1, 2]
[3, 1, 5, 2, 8, 7, 6, 4, 9]
[7, 6, 2, 4, 9, 1, 5, 8, 3]

Verifying Consistency : True
